# Generation: Stuffing Documents

In [1]:
import sys
sys.path.append("..")
import sks_config
# sks_config.SKS_OPENAI_API_KEY   

In [2]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [3]:
vectorstore = Chroma(persist_directory = "./sks-chromadb", 
                     embedding_function = OpenAIEmbeddings(api_key = sks_config.SKS_OPENAI_API_KEY, 
                                                           model='text-embedding-ada-002'))

/var/folders/x8/z943tjhx24786cfv3fmcxq0w0000gn/T/ipykernel_86309/1686351451.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory = "./sks-chromadb",


In [4]:
len(vectorstore.get()['documents'])

66

In [5]:
retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [6]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [7]:
chat = ChatOpenAI(api_key = sks_config.SKS_OPENAI_API_KEY,
                  model_name = 'gpt-4o-mini', 
                  model_kwargs = {'seed': 365},
                  temperature = 0,
                  max_completion_tokens = 250)

/opt/anaconda3/envs/LangChain_course_env1/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [8]:
question = "What software do data scientists use?"

In [9]:
chain = {'context': retriever, 
         'question': RunnablePassthrough()} | prompt_template

In [10]:
chain.invoke(question)

StringPromptValue(text="\nAnswer the following question:\nWhat software do data scientists use?\n\nTo answer the question, use only the following context:\n[Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'As you have probably guessed, analytics generally'}, page_content='So, let’s move on! Programming Languages & Software Employed in Data Science - All the Tools You Need Alright! So… How are the techniques used in data, business intelligence, or predictive analytics applied in real life? Certainly, with the help of computers. You can basically split the relevant tools into two categories—programming languages and software. Knowing a programming language enables you to devise programs that can execute specific operations'), Document(metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'As you have probably guessed, analytics generally'}, page_content='Moreover, you can reuse these programs whenever you need to 

In [11]:
print("\nAnswer the following question:\nWhat software do data scientists use?\n\nTo answer the question, use only the following context:\n[Document(page_content='As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='It’s actually a software framework which was designed to address the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching!', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'})]\n\nAt the end of the response, specify the name of the lecture this context is taken from in the format:\nResources: *Lecture Title*\nwhere *Lecture Title* should be substituted with the title of all resource lectures.\n")


Answer the following question:
What software do data scientists use?

To answer the question, use only the following context:
[Document(page_content='As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='It’s actually a software framework which was designed to address the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers

In [12]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template
         | chat
         | StrOutputParser())

In [13]:
chain.invoke(question)

'Data scientists use a variety of software and programming languages to perform their tasks. The two most popular programming languages in data science are R and Python, which are favored for their ability to manipulate data and their integration with multiple data science software platforms. Additionally, there is a range of software designed for working with big data, including Apache Hadoop, Apache Hbase, and MongoDB, with Hadoop being particularly notable as a collection of programs for big data processing.\n\nResources: *As you have probably guessed, analytics generally*'

In [14]:
print("\nAnswer the following question:\nWhat software do data scientists use?\n\nTo answer the question, use only the following context:\n[Document(page_content='As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='It’s actually a software framework which was designed to address the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching!', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'})]\n\nAt the end of the response, specify the name of the lecture this context is taken from in the format:\nResources: *Lecture Title*\nwhere *Lecture Title* should be substituted with the title of all resource lectures.\n")


Answer the following question:
What software do data scientists use?

To answer the question, use only the following context:
[Document(page_content='As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='It’s actually a software framework which was designed to address the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers